## Consigne 4 avec un multilayer Perceptron sklearn

Admis / non admin

## Librairies

In [ ]:
import pickle
import cv2
from sklearn.neural_network import MLPClassifier
from sklearn.utils import shuffle

## Algorithme

In [ ]:
with open('../data/visages.pkl', 'rb') as fh:
    visages = pickle.load(fh)

with open('../data/noms.pkl', 'rb') as fh:
    noms = pickle.load(fh)

print('Shape of visages matrix --> ', visages.shape)

In [ ]:
N = len(noms)

# ici on mélange les données des visages du dataset afin que l'entrainement soit homogène
visages = visages.reshape(N, -1)
visages = shuffle(visages, random_state = 42)
noms = shuffle(noms, random_state = 42)

# ici on définit notre modèle avec les paramètres souhaités
MLP = MLPClassifier(hidden_layer_sizes=(100, 80, 60, 40, 20), solver='lbfgs')

# entraienment du modèle
MLP.fit(visages, noms)

## Exécution

In [ ]:
cascade_visage = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

camera = cv2.VideoCapture(0) # 0 pour 'built-in' caméra, 1 pour caméra externe

while True:
    
    ret, trame = camera.read()
    if ret == True:
        
        gris = cv2.cvtColor(trame, cv2.COLOR_BGR2GRAY)
        coordonnees_visage = cascade_visage.detectMultiScale(gris, 1.3, 5)

        for (x, y, l, h) in coordonnees_visage:
            
            visage = trame[y:y + h, x:x + l, :]
            visage_redimensionne = cv2.resize(visage, (50, 50)).flatten().reshape(1,-1)
            
            texte = MLP.predict(visage_redimensionne)
            
            cv2.putText(trame, texte[0], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            cv2.rectangle(trame, (x, y), (x + l, y + l), (0, 0, 255), 2)

        cv2.imshow('Reconnaissance faciale en temps réel', trame)
        
        if cv2.waitKey(1) == 27:
            break
            
    else:
        print("erreur")
        break

cv2.destroyAllWindows()
camera.release()